* We used the Profile Report function to have a general idea of what the data contains.
*  We found that we only have one numerical column and one integer column. All columns have high cardinality.
* The first approach we used the LightBGM as baseline model to predict the new data with  24 dropped  columns that have more than 20 percent missing data (na) and we also dropped the remaining rows that contain zero values.
* For the second approach we used the LightBGM a baseline model to predict the new data with  24 dropped  columns that have more than 20 percent missing data (na) and we imputed the remaining nas with the median of the column. The results were not very different.
* Lastly we used the Random forest classifier to predict on the new data with 24 columns dropped for having more than 20% missing values, imputed the remaining row nas and dropped the columns with 50% zero values. 
* After considering all the results we decided to use the data without the columns with more than 50% zero values, more than 20% nas, and without imputed values on the remain row nas.


In [79]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import lightgbm as lgb
from sklearn.metrics import classification_report
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier 

# ConocoPhillips Challenge
### Predict the surface abd down-hole failures using data set provided:




In [80]:
equip = pd.read_csv('equip_failures_training_set.csv', index_col = 'id')

In [81]:
target = equip['target']
equip = equip.drop('target', axis = 1)
equip.head(100)

,sensor1_measure,sensor2_measure,sensor3_measure,sensor4_measure,sensor5_measure,sensor6_measure,sensor7_histogram_bin0,sensor7_histogram_bin1,sensor7_histogram_bin2,sensor7_histogram_bin3,...,sensor105_histogram_bin2,sensor105_histogram_bin3,sensor105_histogram_bin4,sensor105_histogram_bin5,sensor105_histogram_bin6,sensor105_histogram_bin7,sensor105_histogram_bin8,sensor105_histogram_bin9,sensor106_measure,sensor107_measure
id,,,,,,,,,,,,,,,,,,,,,
1,76698,na,2130706438,280,0,0,0,0,0,0,...,1240520,493384,721044,469792,339156,157956,73224,0,0,0
2,33058,na,0,na,0,0,0,0,0,0,...,421400,178064,293306,245416,133654,81140,97576,1500,0,0
3,41040,na,228,100,0,0,0,0,0,0,...,277378,159812,423992,409564,320746,158022,95128,514,0,0
4,12,0,70,66,0,10,0,0,0,318,...,240,46,58,44,10,0,0,0,4,32
5,60874,na,1368,458,0,0,0,0,0,0,...,622012,229790,405298,347188,286954,311560,433954,1218,0,0
6,38312,na,2130706432,218,0,0,0,0,0,0,...,388574,288278,900430,300412,1534,338,856,0,0,0
7,14,0,6,na,0,0,0,0,0,0,...,168,48,60,28,0,0,0,0,0,0
8,102960,na,2130706432,116,0,0,0,0,0,0,...,715518,384948,915978,1052166,1108672,341532,129504,7832,0,0
9,78696,na,0,na,0,0,0,0,0,0,...,699290,362510,1190028,1012704,160090,63216,41202,4,0,0


In [82]:
from pandas_profiling import ProfileReport
ProfileReport(equip, check_correlation=True)

Number of variables,171
Number of observations,60000
Total Missing (%),0.0%
Total size in memory,78.3 MiB
Average record size in memory,1.3 KiB
Numeric,2
Categorical,169
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [83]:
missing = (equip.replace('na',np.nan).isna().sum().values/60000) * 100


In [84]:
percent_missing = pd.Series(missing.round(2), index = equip.columns) 

In [85]:
percent_missing

sensor1_measure              0.00
sensor2_measure             77.22
sensor3_measure              5.56
sensor4_measure             24.77
sensor5_measure              4.17
sensor6_measure              4.17
sensor7_histogram_bin0       1.12
sensor7_histogram_bin1       1.12
sensor7_histogram_bin2       1.12
sensor7_histogram_bin3       1.12
sensor7_histogram_bin4       1.12
sensor7_histogram_bin5       1.12
sensor7_histogram_bin6       1.12
sensor7_histogram_bin7       1.12
sensor7_histogram_bin8       1.12
sensor7_histogram_bin9       1.12
sensor8_measure              1.08
sensor9_measure              1.05
sensor10_measure             1.05
sensor11_measure             7.33
sensor12_measure             1.07
sensor13_measure             1.05
sensor14_measure             1.07
sensor15_measure             0.98
sensor16_measure             1.07
sensor17_measure             0.98
sensor18_measure             4.54
sensor19_measure             1.05
sensor20_measure             1.05
sensor21_measu

In [86]:
drop_20upmissing = percent_missing[percent_missing > 20].index # what percentage of nas per column
drop_20upmissing

Index(['sensor2_measure', 'sensor4_measure', 'sensor36_measure',
       'sensor37_measure', 'sensor38_measure', 'sensor39_measure',
       'sensor40_measure', 'sensor41_measure', 'sensor42_measure',
       'sensor43_measure', 'sensor56_measure', 'sensor57_measure',
       'sensor58_measure', 'sensor65_measure', 'sensor68_measure',
       'sensor70_measure', 'sensor71_measure', 'sensor72_measure',
       'sensor73_measure', 'sensor74_measure', 'sensor75_measure',
       'sensor76_measure', 'sensor77_measure', 'sensor78_measure'],
      dtype='object')

In [11]:
sum(equip.replace('na',np.nan).isna().sum()) # number of nas in total

850015

In [12]:
missing = equip.replace('na',np.nan).isna().any()
missing_cols = list(missing[missing].index)

In [13]:
print(len(missing_cols))

169


In [14]:
na_col = equip.replace('na',np.nan).isna().sum()

In [15]:
na_col[na_col>0].shape

(169,)

In [16]:
na_col[na_col > 3000].shape # 42 columns have more than 3000 missing values

(42,)

## Dropping Column with > 20% missing data

In [17]:
equip = pd.read_csv('equip_failures_training_set.csv', index_col = 'id')
equip = equip.replace('na',np.nan)
equip = equip.drop(drop_20upmissing, axis = 1)
equip = equip.dropna(axis =0)

In [18]:
equip.shape

(47865, 147)

In [19]:
target = equip['target']
equip = equip.drop('target', axis = 1)

In [20]:
baseline_data = equip
baseline_data.shape

(47865, 146)

In [21]:
sum(baseline_data.replace('na',np.nan).isna().sum())

0

In [22]:
my_cols = list(set(missing_cols) - set(drop_20upmissing))

In [23]:
baseline_data.dtypes

sensor1_measure              int64
sensor3_measure             object
sensor5_measure             object
sensor6_measure             object
sensor7_histogram_bin0      object
sensor7_histogram_bin1      object
sensor7_histogram_bin2      object
sensor7_histogram_bin3      object
sensor7_histogram_bin4      object
sensor7_histogram_bin5      object
sensor7_histogram_bin6      object
sensor7_histogram_bin7      object
sensor7_histogram_bin8      object
sensor7_histogram_bin9      object
sensor8_measure             object
sensor9_measure             object
sensor10_measure            object
sensor11_measure            object
sensor12_measure            object
sensor13_measure            object
sensor14_measure            object
sensor15_measure            object
sensor16_measure            object
sensor17_measure            object
sensor18_measure            object
sensor19_measure            object
sensor20_measure            object
sensor21_measure            object
sensor22_measure    

In [24]:
floats = baseline_data.select_dtypes(object).apply(pd.to_numeric,downcast = 'float') # converting object dataframe to floats

In [25]:
baseline_data[floats.columns] = floats.values

In [26]:
baseline_data.head()

,sensor1_measure,sensor3_measure,sensor5_measure,sensor6_measure,sensor7_histogram_bin0,sensor7_histogram_bin1,sensor7_histogram_bin2,sensor7_histogram_bin3,sensor7_histogram_bin4,sensor7_histogram_bin5,...,sensor105_histogram_bin2,sensor105_histogram_bin3,sensor105_histogram_bin4,sensor105_histogram_bin5,sensor105_histogram_bin6,sensor105_histogram_bin7,sensor105_histogram_bin8,sensor105_histogram_bin9,sensor106_measure,sensor107_measure
id,,,,,,,,,,,,,,,,,,,,,
1,76698,2.130706e+09,0.0,0.0,0.0,0.0,0.0,0.0,37250.0,1432864.0,...,1240520.0,493384.0,721044.0,469792.0,339156.0,157956.0,73224.0,0.0,0.0,0.0
2,33058,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,18254.0,653294.0,...,421400.0,178064.0,293306.0,245416.0,133654.0,81140.0,97576.0,1500.0,0.0,0.0
3,41040,2.280000e+02,0.0,0.0,0.0,0.0,0.0,0.0,1648.0,370592.0,...,277378.0,159812.0,423992.0,409564.0,320746.0,158022.0,95128.0,514.0,0.0,0.0
4,12,7.000000e+01,0.0,10.0,0.0,0.0,0.0,318.0,2212.0,3232.0,...,240.0,46.0,58.0,44.0,10.0,0.0,0.0,0.0,4.0,32.0
5,60874,1.368000e+03,0.0,0.0,0.0,0.0,0.0,0.0,43752.0,1966618.0,...,622012.0,229790.0,405298.0,347188.0,286954.0,311560.0,433954.0,1218.0,0.0,0.0


In [27]:
X_train, X_valid, y_train, y_valid = train_test_split(baseline_data,target, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0) # spplitting data to train and validate

In [28]:
dtrain = lgb.Dataset(X_train, label= y_train)
dvalid = lgb.Dataset(X_valid, label= y_valid)

param = {'num_leaves': 30, 'objective': 'binary'}
param['metric'] = 'f1_metric'
num_round = 1000

bst = lgb.train(param, dtrain, num_round, valid_sets=[dvalid], verbose_eval= False) # lightgbm model (tree model)



https://ragrawal.wordpress.com/2019/04/03/f1-score/6

In [29]:

y_pred = bst.predict(X_valid)
valid_score = f1_score(y_valid, y_pred.round(), average = 'micro')
print(f" F-1 score: {valid_score:.4f}") # mean F1 Score

 F-1 score: 0.9845


In [30]:
print(classification_report(y_valid, y_pred.round())) # need to interpret correctly

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      9497
           1       0.00      0.00      0.00        76

   micro avg       0.98      0.98      0.98      9573
   macro avg       0.50      0.50      0.50      9573
weighted avg       0.98      0.98      0.98      9573



In [31]:

baseline_data.to_csv('NO_NAs_NO20UPmissing.csv', index=False) # writting file to txt file

## SIMPLE IMPUTER, Dropped columns 20 up and imputed missing rows

In [32]:
 # instead of dropping the rows nas we imputed with median
equip = pd.read_csv('equip_failures_training_set.csv', index_col = 'id')
equip = equip.replace('na',np.nan)
equip = equip.drop(drop_20upmissing, axis = 1) # dropping collumns with more than 20% na values

target = equip['target']
equip = equip.drop('target', axis = 1)



In [33]:
baseline_2 = equip
baseline_2.shape

(60000, 146)

In [34]:
missing_2 = baseline_2.isnull().sum() # checking how many na values per column
missing_2.head()

sensor1_measure              0
sensor3_measure           3335
sensor5_measure           2500
sensor6_measure           2500
sensor7_histogram_bin0     671
dtype: int64

In [35]:
ints = baseline_2.select_dtypes(object).apply(pd.to_numeric,downcast = 'signed') #converting object dataframe to integer
baseline_2[ints.columns] = ints.values

In [36]:
baseline_2.isnull().sum().head() # checking missing per column

sensor1_measure              0
sensor3_measure           3335
sensor5_measure           2500
sensor6_measure           2500
sensor7_histogram_bin0     671
dtype: int64

In [37]:
missing_2  = list(missing_2[missing_2 >0].index) # storing the columns that have missing data

In [38]:
# Make new columns indicating what will be imputed optinal
#for col in missing_2:
   # baseline_2[col + '_was_missing'] = baseline_2[col].isnull()

In [39]:
baseline_2.head() # checking data

,sensor1_measure,sensor3_measure,sensor5_measure,sensor6_measure,sensor7_histogram_bin0,sensor7_histogram_bin1,sensor7_histogram_bin2,sensor7_histogram_bin3,sensor7_histogram_bin4,sensor7_histogram_bin5,...,sensor105_histogram_bin2,sensor105_histogram_bin3,sensor105_histogram_bin4,sensor105_histogram_bin5,sensor105_histogram_bin6,sensor105_histogram_bin7,sensor105_histogram_bin8,sensor105_histogram_bin9,sensor106_measure,sensor107_measure
id,,,,,,,,,,,,,,,,,,,,,
1,76698,2.130706e+09,0.0,0.0,0.0,0.0,0.0,0.0,37250.0,1432864.0,...,1240520.0,493384.0,721044.0,469792.0,339156.0,157956.0,73224.0,0.0,0.0,0.0
2,33058,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,18254.0,653294.0,...,421400.0,178064.0,293306.0,245416.0,133654.0,81140.0,97576.0,1500.0,0.0,0.0
3,41040,2.280000e+02,0.0,0.0,0.0,0.0,0.0,0.0,1648.0,370592.0,...,277378.0,159812.0,423992.0,409564.0,320746.0,158022.0,95128.0,514.0,0.0,0.0
4,12,7.000000e+01,0.0,10.0,0.0,0.0,0.0,318.0,2212.0,3232.0,...,240.0,46.0,58.0,44.0,10.0,0.0,0.0,0.0,4.0,32.0
5,60874,1.368000e+03,0.0,0.0,0.0,0.0,0.0,0.0,43752.0,1966618.0,...,622012.0,229790.0,405298.0,347188.0,286954.0,311560.0,433954.0,1218.0,0.0,0.0


In [40]:
X_train, X_valid, y_train, y_valid = train_test_split(baseline_2,target, # splitting the data to train and valid
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

In [41]:
# Imputation
my_imputer = SimpleImputer(strategy = 'median')
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

In [42]:
imputed_X_valid.isnull().sum().head() # checking that there are no missing values

sensor1_measure           0
sensor3_measure           0
sensor5_measure           0
sensor6_measure           0
sensor7_histogram_bin0    0
dtype: int64

In [43]:
dtrain = lgb.Dataset(imputed_X_train, label= y_train) # training the model again
dvalid = lgb.Dataset(imputed_X_valid, label= y_valid)

param = {'num_leaves': 30, 'objective': 'binary'}
param['metric'] = 'f1_metric'
num_round = 1000

bst = lgb.train(param, dtrain, num_round, valid_sets=[dvalid], verbose_eval= False)



In [44]:
y_pred = bst.predict(imputed_X_valid) # we got similar results
valid_score = f1_score(y_valid, y_pred.round(), average = 'micro')
print(f" F-1 score: {valid_score:.4f}")

 F-1 score: 0.9951


In [45]:
print(classification_report(y_valid, y_pred.round()))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     11816
           1       0.90      0.76      0.83       184

   micro avg       1.00      1.00      1.00     12000
   macro avg       0.95      0.88      0.91     12000
weighted avg       0.99      1.00      0.99     12000



In [46]:
dicti = dict() # creating a dictionary to store the percent of zeros per columns

for col in baseline_2:
    dicti[col] = np.count_nonzero(baseline_2[col])
    
    

In [47]:
non_zeros = pd.Series(dicti)

In [48]:
zeros_percent = (((60000 - non_zeros.values)/60000)*100).round(2)

In [49]:
zeros_percent = pd.Series(zeros_percent,index = non_zeros.index)


In [50]:
len(zeros_percent[zeros_percent > 50]) # checking how many columns have more than 50% zero values

50

In [51]:
col_with_0 = list(zeros_percent[zeros_percent > 50].index) # storing the name of the columns with > 50% zeros

In [67]:
col_with_0

['sensor5_measure',
 'sensor6_measure',
 'sensor7_histogram_bin0',
 'sensor7_histogram_bin1',
 'sensor7_histogram_bin2',
 'sensor7_histogram_bin3',
 'sensor7_histogram_bin9',
 'sensor9_measure',
 'sensor10_measure',
 'sensor11_measure',
 'sensor12_measure',
 'sensor13_measure',
 'sensor18_measure',
 'sensor19_measure',
 'sensor20_measure',
 'sensor21_measure',
 'sensor24_histogram_bin0',
 'sensor24_histogram_bin1',
 'sensor24_histogram_bin2',
 'sensor24_histogram_bin3',
 'sensor24_histogram_bin4',
 'sensor24_histogram_bin5',
 'sensor24_histogram_bin9',
 'sensor25_histogram_bin7',
 'sensor25_histogram_bin8',
 'sensor25_histogram_bin9',
 'sensor26_histogram_bin9',
 'sensor60_measure',
 'sensor62_measure',
 'sensor64_histogram_bin0',
 'sensor64_histogram_bin1',
 'sensor64_histogram_bin2',
 'sensor69_histogram_bin9',
 'sensor81_measure',
 'sensor82_measure',
 'sensor83_measure',
 'sensor84_measure',
 'sensor85_measure',
 'sensor86_measure',
 'sensor87_measure',
 'sensor88_measure',
 'senso

In [52]:
baseline_3 = baseline_2.drop(col_with_0, axis = 1) # dropping columns defined above

In [53]:
baseline_3.isnull().sum().head() # data without columns with more than 20% nas and > 50% zeros

sensor1_measure              0
sensor3_measure           3335
sensor7_histogram_bin4     671
sensor7_histogram_bin5     671
sensor7_histogram_bin6     671
dtype: int64

In [54]:
X_train_3, X_valid_3, y_train_3, y_valid_3 = train_test_split(baseline_3,target, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0) # splitting the data to train & validate

In [55]:
# Imputation
my_imputer_3 = SimpleImputer(strategy = 'median')
imputed_X_train_3 = pd.DataFrame(my_imputer.fit_transform(X_train_3))
imputed_X_valid_3 = pd.DataFrame(my_imputer.transform(X_valid_3))

# Imputation removed column names; put them back
imputed_X_train_3.columns = X_train_3.columns
imputed_X_valid_3.columns = X_valid_3.columns # imputing missing values for the new dataset

In [56]:
dtrain_3 = lgb.Dataset(imputed_X_train_3, label= y_train)  # training data
dvalid_3 = lgb.Dataset(imputed_X_valid_3, label= y_valid)


param = {'num_leaves': 30, 'objective': 'binary'}
param['metric'] = 'f1_metric'
num_round = 1000

bst_3= lgb.train(param, dtrain_3, num_round, valid_sets=[dvalid_3], verbose_eval= False)



In [57]:
y_pred = bst.predict(X_valid)
valid_score = f1_score(y_valid, y_pred.round(), average = 'micro')
print(f" F-1 score: {valid_score:.4f}")

 F-1 score: 0.9949


In [58]:
baseline_3.shape

(60000, 96)

## Random Forest Classifier

In [59]:
model = RandomForestClassifier(random_state=1)

In [60]:
model.fit(imputed_X_train_3, y_train_3)
preds = model.predict(imputed_X_valid_3)

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [61]:
valid_score = f1_score(y_valid_3, preds.round(), average = 'micro')
print(f" F-1 score: {valid_score:.4f}")

 F-1 score: 0.9918


## Feature Selection

In [62]:
from sklearn.feature_selection import SelectKBest, f_classif


# Keep 5 features
selector = SelectKBest(f_classif, k=5)

X_new = selector.fit_transform(imputed_X_train_3,y_train_3)
X_new

/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [52] are constant.
  UserWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


array([[6.06600000e+04, 3.48586000e+06, 6.06605700e+04, 3.48586000e+06,
        2.95686624e+06],
       [3.41200000e+03, 2.14304000e+05, 3.41297000e+03, 2.14304000e+05,
        1.52820480e+05],
       [2.96640000e+04, 2.49231600e+06, 2.96640400e+04, 2.49231600e+06,
        1.99385664e+06],
       ...,
       [5.62564000e+05, 5.35024000e+07, 5.62564000e+05, 5.35024000e+07,
        3.31132000e+07],
       [1.61800000e+03, 4.40136000e+05, 1.61724000e+03, 4.40136000e+05,
        1.07092800e+05],
       [1.42000000e+03, 1.39878000e+05, 1.42037000e+03, 1.39878000e+05,
        7.40774400e+04]])

In [63]:
# Set the regularization parameter C=1
logistic = LogisticRegression(C=1, penalty="l1", random_state=7).fit(imputed_X_train_3, y_train_3)
model = SelectFromModel(logistic, prefit=True)

X_new = model.transform(imputed_X_train_3)
X_new

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


array([[6.06600e+04, 2.70000e+02, 9.40000e+01, ..., 1.40900e+04,
        1.11926e+03, 1.18000e+03],
       [3.41200e+03, 3.80000e+01, 1.20000e+01, ..., 8.02000e+02,
        7.57780e+02, 8.34000e+02],
       [2.96640e+04, 0.00000e+00, 0.00000e+00, ..., 1.13840e+04,
        7.57780e+02, 8.34000e+02],
       ...,
       [5.62564e+05, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [1.61800e+03, 2.52000e+02, 9.80000e+01, ..., 9.36000e+02,
        5.00000e-01, 4.60000e+01],
       [1.42000e+03, 1.30000e+02, 4.60000e+01, ..., 4.76000e+02,
        1.04420e+02, 7.00000e+01]])

In [64]:
selected_features = pd.DataFrame(model.inverse_transform(X_new), 
                                 index=imputed_X_train_3.index,
                                 columns=imputed_X_train_3.columns)

# Dropped columns have values of all 0s, keep other columns 
selected_columns = selected_features.columns[selected_features.var() != 0]

In [65]:
selected_columns = list(selected_columns)

In [66]:
selected_columns

['sensor1_measure',
 'sensor22_measure',
 'sensor23_measure',
 'sensor28_measure',
 'sensor29_measure',
 'sensor31_measure',
 'sensor45_measure',
 'sensor49_measure',
 'sensor63_measure',
 'sensor69_histogram_bin0',
 'sensor69_histogram_bin1',
 'sensor69_histogram_bin8',
 'sensor91_measure',
 'sensor95_measure',
 'sensor103_measure',
 'sensor104_measure']

In [71]:
selected_columns = pd.Series(selected_columns)

In [72]:
selected_columns.to_csv('feature_selection_using_selectKbest', index=False)